<div align='center'><img src='imgs/logos.png' alt='Training entities logos' width='80%'></img></div>

<hr>

# FPCUP - Working Group Africa

# Training: 
# Detecting plastic pollution and events of marine debris pollution using Sentinel -2

Emanuel Castanho (AIR Centre)

25 OCTOBER 2023

<hr>

### Install libraries

In [ ]:
# !pip install --find-links=https://girder.github.io/large_image_wheels --no-cache GDAL==3.8.0 --quiet
# print("GDAL checked!")
# !pip install localtileserver==0.7.2 --quiet
# print("localtileserver checked!")
# !pip install geopandas==0.14.0 --quiet
# print("geopandas checked!")
# !pip install leafmap==0.27.0 --quiet
# print("leafmap checked!")
# !pip install scikit-learn==1.1.1 --quiet # To load the model you need the same version
# print("scikit-learn checked!")
# !pip install pyarrow==13.0.0 --quiet
# print("pyarrow checked!")

In [ ]:
# !jupyter nbextension install --py --symlink --sys-prefix ipyleaflet
# !jupyter nbextension enable --py --sys-prefix ipyleaflet

### Import libraries

In [1]:
import os
import glob
import leafmap
from osgeo import gdal
import pickle as pkl
import numpy as np
import pandas as pd
from ipyleaflet import CircleMarker, LayerGroup
from pyproj import Transformer

### User inputs

In [2]:
# Basepath as string. Empty for current path.
basepath = ""

# Stack file with ACOLITE Rayleigh-corrected bands and spectral indices.
rc_stack = "S2A_MSI_2022_04_13_08_08_18_T36JUN_stack.tif"

### Explore stack

In [3]:
# Stack fullpath and name without extension
stack_fullpath = os.path.join(basepath, "data", rc_stack)
stack_name = rc_stack[:-4]

# Check available bands and their numbering
stack_dataset = gdal.Open(stack_fullpath)
stack_bands = {stack_dataset.GetRasterBand(i).GetDescription(): i for i in range(1, stack_dataset.RasterCount+1)}
print("Available bands (RC Bands + Spectral Indices) inside stack:")
print(stack_bands)

Available bands (RC Bands + Spectral Indices) inside stack:
{'B01': 1, 'B02': 2, 'B03': 3, 'B04': 4, 'B05': 5, 'B06': 6, 'B07': 7, 'B08': 8, 'B8A': 9, 'B11': 10, 'B12': 11, 'NDVI': 12, 'FAI': 13, 'FDI': 14, 'SI': 15, 'NDWI': 16, 'NRD': 17, 'NDMI': 18, 'BSI': 19}


### Start leafmap and add data

In [4]:
# Start basemap
m = leafmap.Map(location=[31.4710, -29.6108], zoom_start=5)

# Add RGB (Red-Green-Blue) layer 
m.add_raster(stack_fullpath, band=[4, 3, 2], vmin=0, vmax=0.15, layer_name="RGB")

# Add NDWI (Normalized Difference Water Index)
ndwi_vis = {"cmap":"bwr_r", "vmin":-1, "vmax":0.8, "name":"NDWI"}
m.add_raster(stack_fullpath, band=16, cmap=ndwi_vis["cmap"], vmin=ndwi_vis["vmin"], vmax=ndwi_vis["vmax"], layer_name=ndwi_vis["name"])
m.add_colormap(cmap=ndwi_vis["cmap"], label=ndwi_vis["name"], vmin=ndwi_vis["vmin"], vmax=ndwi_vis["vmax"], width=4)

# Add FDI (Floating Debris Index)
fdi_vis = {"cmap":"twilight_shifted", "vmin":0, "vmax":0.1, "name":"FDI"}
m.add_raster(stack_fullpath, band=14, cmap=fdi_vis["cmap"], vmin=fdi_vis["vmin"], vmax=fdi_vis["vmax"], layer_name=fdi_vis["name"])
m.add_colormap(cmap=fdi_vis["cmap"], label=fdi_vis["name"], vmin=fdi_vis["vmin"], vmax=fdi_vis["vmax"], width=4)

### Convert stack to dataframe

In [5]:
# All the stack bands (RC Bands + Spectral Indices) will be Machine Learning (ML) model features
ml_model_features = list(stack_bands.keys())
 
# Initiate the loop with B01
init_stack_data = stack_dataset.GetRasterBand(1).ReadAsArray()
# Reshape as single column
stack_data = init_stack_data.reshape(-1,1)

# Loop through the other bands
for band in ml_model_features[1:]:
    # Get data
    band_data = stack_dataset.GetRasterBand(stack_bands[band]).ReadAsArray()
    band_data_reshaped = band_data.reshape(-1,1)
    stack_data = np.concatenate([stack_data, band_data_reshaped], axis=1)

# Shape to use in reshape
band_shape = init_stack_data.shape

# Global dataframe for classification
global_cl_df = pd.DataFrame(stack_data, columns=ml_model_features)

# Dataframe for classification, without not a numbers. RF model does not work with NaN
no_nans_cl_df = global_cl_df.dropna(axis=0, how='any')

print(no_nans_cl_df.head())

        B01       B02       B03       B04       B05       B06       B07  \
0  0.089779  0.085006  0.066746  0.060497  0.059147  0.057825  0.059238   
1  0.089780  0.079840  0.066410  0.060074  0.059147  0.057825  0.059238   
2  0.089780  0.080201  0.067641  0.059227  0.058833  0.060110  0.059857   
3  0.089780  0.082604  0.067641  0.060074  0.058833  0.060110  0.059857   
4  0.089780  0.082243  0.066634  0.059756  0.058833  0.060110  0.059548   

        B08       B8A       B11       B12      NDVI       FAI       FDI  \
0  0.055262  0.053458  0.040377  0.033586 -0.045223 -0.001669  0.015881   
1  0.055365  0.053458  0.040377  0.033586 -0.040792 -0.001218  0.015983   
2  0.054647  0.055092  0.039675  0.034487 -0.040216 -0.001115  0.016138   
3  0.053520  0.055092  0.039675  0.034487 -0.057697 -0.002939  0.015010   
4  0.052597  0.058461  0.040276  0.035288 -0.063717 -0.003707  0.013452   

         SI      NDWI       NRD      NDMI       BSI  
0  0.929192  0.094121 -0.005235  0.155645 -0

### Load Random Forest ML model 

In [6]:
# The trained model is stored in a pickle file
rf_model_fullpath = glob.glob(os.path.join(basepath, "data", "MARIDA_RF-model", "*.pkl"))[0]
rf_model = pkl.load(open(rf_model_fullpath, 'rb'))
print(rf_model)

RandomForestClassifier(ccp_alpha=0, class_weight='balanced_subsample',
                       max_depth=20, min_impurity_decrease=0,
                       min_weight_fraction_leaf=0, n_estimators=125, n_jobs=-1,
                       oob_score=True, random_state=5)


### Classification

In [7]:
if __name__ == "__main__": # Brokenpipe bug is caused by multiprocessing on Jupyter, does not affect the results
    # Dataframe of 0's to store the classification results
    cl_results_struct_df = pd.DataFrame(0, index=range(0, len(global_cl_df.index)), columns=['ClassNum'])

    # If dataframe for classification without NaNs is empty, then the final classification is only 0.  
    if len(no_nans_cl_df.index) == 0:
        cl_results_flat = np.array(cl_results_struct_df).flatten()
        cl_results_reshape = cl_results_flat.reshape(band_shape)
    else:
        # RF Classification
        cl_results = rf_model.predict(no_nans_cl_df)

        # Index results, construct final dataframe and reshape
        cl_results_indexed = pd.DataFrame(cl_results, index=no_nans_cl_df.index, columns=['ClassNum'])
        cl_results_df = cl_results_indexed.combine_first(cl_results_struct_df)
        cl_results_flat = np.array(cl_results_df).flatten()
        cl_results_reshape = cl_results_flat.reshape(band_shape)

    # Save classification map
    sc_map_fullpath = os.path.join(basepath, "processing", "scmap.tif")
    driver = gdal.GetDriverByName("GTiff")
    sc_raster = driver.Create(sc_map_fullpath, stack_dataset.RasterXSize, stack_dataset.RasterYSize, 1, gdal.GDT_Byte)
    sc_raster.SetProjection(stack_dataset.GetProjectionRef())
    sc_raster.SetGeoTransform(stack_dataset.GetGeoTransform())
    sc_raster_band = sc_raster.GetRasterBand(1)
    sc_raster_band.WriteArray(cl_results_reshape)
    sc_raster = None


### Add classification to leafmap

In [8]:
# Scene classification map
# Color palette:
# 0- No Data - NaNs
# 1- Marine Debris - Floating plastics or other polymers, mixed anthropogenic debris
# 2- Dense Sargassum - Dense floating Sargassum macroalgae
# 3- Sparse Sargassum - Sparse floating Sargassum macroalgae
# 4- Natural Organic Material - Vegetation & Wood
# 5- Ship - Sailing & Anchored Vessels
# 6- Clouds - Clouds including thin Clouds
# 7- Marine Water SC - Marine Water Super Class: Clear Water, Wakes, CloudS, Waves and MixWater
# 8- Sediment-Laden Water - High-Sediment river discharges with brown colour
# 9- Foam - Foam recorded at river fronts or coastal wave breaking area
# 10- Turbid Water - Turbid waters close to coastal areas
# 11- Shallow Water - Coastal waters, including coral reefs and submerged vegetation

cl_vis = {"cmap":["#ffffff", "#ff0000", "#008000", "#32cd32", "#b22222", "#ffa500", "#c0c0c0", "#000080", "#ffd700", "#800080", "#bdb76b", "#00ced1"], "vmin":0, "vmax":11, "name":"Classification"}
m.add_raster(sc_map_fullpath, band=1, cmap=cl_vis["cmap"], vmin=cl_vis["vmin"], vmax=cl_vis["vmax"], layer_name=cl_vis["name"])


### Extract Marine Debris pixels center coordinates

In [9]:
# Convert scene classification GeoTIFF to XYZ
sc_map_raster = gdal.Open(sc_map_fullpath)
xyz_fullpath = os.path.join(basepath, "processing", "scmap.xyz")
xyz_raster = gdal.Translate(xyz_fullpath, sc_map_raster)

# Close rasters
sc_map_raster = None
xyz_raster = None

# Convert XYZ to Feather
xyz_df = pd.read_csv(xyz_fullpath, sep=" ", header=None)
xyz_df.columns = ["CenterX", "CenterY", "Value"]
feather_fullpath = os.path.join(basepath, "processing", "scmap.feather")
xyz_df.to_feather(feather_fullpath)

# Delete intermediate XYZ file
os.remove(xyz_fullpath)

In [10]:
# Function that converts Sentinel-2 CRS to EPSG4326 depending on Tile
def s2_coord_converter(row):
    '''
    This function converts UTM coordinates to EPSG:4326. It uses Sentinel-2 tile information.
    https://sentinels.copernicus.eu/documents/247904/685211/Sentinel-2-Products-Specification-Document.pdf/fb1fc4dc-12ca-4674-8f78-b06efa871ab9
    
    Inputs: row - Row of dataframe, must contain: Tile, CenterX and CenterY.
    Outputs: (lon, lat) - Tuple with longitude and latitude pair.
    '''
    # First letter of tile code
    if row["Tile"][2] in list(map(chr, range(67, 78))):
        # South C-M
        pos = "327"
    elif row["Tile"][2] in list(map(chr, range(78, 89))):
        # North N-X
        pos = "326"
    else:
        # Undefined
        pos = None

    # Conversion of coordinates. EPSG + South or North + first 2 digits of tile code
    in_crs = "epsg:" + pos + row["Tile"][0:2]
    out_crs = "epsg:4326"
    trans = Transformer.from_crs(in_crs, out_crs)
    x, y = row["CenterX"], row["CenterY"]
    lat, lon = trans.transform(x, y)
    
    return (lon, lat)

In [11]:
# Filter only MD and convert coordinate system
feather_df = pd.read_feather(feather_fullpath)
feather_md_df = feather_df[feather_df.Value==1]
feather_md_df["Tile"] = stack_name.split("_")[8][1:]
feather_md_df["CenterLon"], feather_md_df["CenterLat"] = zip(*(feather_md_df.apply(lambda row: s2_coord_converter(row), axis=1)))
print(feather_md_df.head())

       CenterX  CenterY  Value   Tile  CenterLon  CenterLat
1351    357135  6733095      1  36JUN  31.525806 -29.523608
23690   356535  6732965      1  36JUN  31.519599 -29.524712
29043   358375  6732935      1  36JUN  31.538577 -29.525193
29044   358385  6732935      1  36JUN  31.538680 -29.525194
29047   358415  6732935      1  36JUN  31.538989 -29.525197


### Plot suspected MD locations

In [12]:
# Suspected Marine Debris as black dots
markers = []
for idx, row in feather_md_df.iterrows():
    marker = CircleMarker(location=[row["CenterLat"], row["CenterLon"]], radius=1, color="black", fill_color="black", opacity=1)
    markers.append(marker)
layer_group = LayerGroup(layers=markers, name="Suspected MD")
m.add_layer(layer_group)

### Show final map

In [13]:
m

Map(center=[-29.607231292818643, 31.473917956217676], controls=(ZoomControl(options=['position', 'zoom_in_text…